# Multi-Product Cross-Sell Model - AutoML Training

## Overview
This notebook trains 4 independent classification models to predict cross-sell opportunities for different product categories. Each model uses a systematically selected feature set optimized for that product's unique characteristics and drivers.

## Feature Selection Methodology
Features are selected using a data-driven approach combining:
1. **Statistical Analysis (70% weight)**: Correlation with target variable
2. **Domain Knowledge (30% weight)**: Business relevance and product-specific drivers
3. **Optimization**: 25-50 features per model to balance signal capture and model complexity

## Target Products
- **Investment Products** (64.2% of cross-sells)
- **Retirement Products** (25.8% of cross-sells)
- **Life Insurance** (5.4% of cross-sells)
- **Network Products** (4.6% of cross-sells)

## Selected Features by Product

### Investment Products (30-40 features)
**Primary Drivers**: Wealth capacity, risk appetite, market timing

Key features:
- Wealth indicators: `acct_val_amt`, `aum_segment`, `wc_total_assets`
- Risk profile: `wc_assetmix_stocks`, `aggressive_investor`, `wc_assetmix_mutual_funds`
- Market timing: `snp_close_lead_6`, `snp_close_variation`
- Demographics: `client_age`, `channel`, `client_tenure_years`
- Life stage: `wealth_building_trigger`
- Affinity: `advisor_investment_affinity`

### Retirement Products (30-35 features)
**Primary Drivers**: Life stage, conservative profile, long-term planning

Key features:
- Life stage: `client_age`, `retirement_planning_trigger`
- Financial capacity: `acct_val_amt`, `aum_segment`, `monthly_preminum_amount`
- Conservative profile: `wc_assetmix_bonds`, `wc_assetmix_deposits`, `conservative_investor`
- Long-term context: `snp_close_lead_12`
- Relationship: `channel`, `client_tenure_years`
- Affinity: `branch_retirement_affinity`

### Life Insurance (25-30 features)
**Primary Drivers**: Family protection needs, payment capacity, life stage

Key features:
- Life stage: `client_age`, `family_protection_trigger`
- Financial responsibility: `face_amt`, `monthly_preminum_amount`, `acct_val_amt`
- Existing coverage: `cash_val_amt`
- Risk awareness: `snp_close_variation`
- Relationship: `channel`, `client_tenure_years`

### Network Products (20-25 features)
**Primary Drivers**: Ultra-high wealth, advisor relationship, sophistication

Key features:
- Ultra-high wealth: `acct_val_amt`, `aum_segment`, `wc_total_assets`
- Relationship strength: `channel`, `client_tenure_years`, `advisor_investment_affinity`
- Sophistication: `wc_assetmix_stocks`, `aggressive_investor`
- Demographics: `client_age`

## Expected Training Time
6-8 hours total (1.5-2 hours per model with 120-minute timeout)


In [0]:
# Configuration
dbutils.widgets.text("target_schema", "eda_smartlist.us_wealth_management_smartlist")
dbutils.widgets.text("business_month", "202510")
dbutils.widgets.text("automl_timeout_minutes", "60")
dbutils.widgets.text("automl_sample_size", "100000")
# Get parameters
target_schema = dbutils.widgets.get("target_schema")
business_month = dbutils.widgets.get("business_month")
automl_sample_size = int(dbutils.widgets.get("automl_sample_size"))
print(f"Target Schema: {target_schema}")
print(f"Business Month: {business_month}")
print(f"AutoML Sample Size: {automl_sample_size:,} records")

Target Schema: eda_smartlist.us_wealth_management_smartlist
Business Month: 202510
AutoML Sample Size: 100,000 records


## Configuration
Set training parameters and schema configuration


In [0]:
# Import Libraries
from databricks import automl
import mlflow
from mlflow.tracking import MlflowClient
from datetime import datetime
from pyspark.sql.functions import col, when, abs as spark_abs
from pyspark.sql.types import DecimalType, StructType, StructField, StringType, IntegerType, DoubleType
import pandas as pd
import sys

# Import our custom modules for robust architecture
sys.path.append('/Workspace/Users/juan.hernandez@equitable.com/multi_product_model')
from feature_engineering import prepare_training_data, validate_target_columns, get_interaction_feature_names
from config import create_default_config
from validation import validate_data_quality, validate_target_distribution, validate_model_inputs

print("Libraries and custom modules imported successfully")

# Initialize configuration
config = create_default_config(target_schema)
print(f"Configuration initialized for schema: {target_schema}")
print(f"AutoML timeout: {config.automl_timeout_minutes} minutes")
print(f"Sample size: {config.automl_sample_size:,} records")


(2025-11-26 14:49:52) WARNING: Hyperopt is deprecated for Databricks runtime for machine learning and will not be pre-installed in the next major version.


Libraries and custom modules imported successfully
Configuration initialized for schema: eda_smartlist.us_wealth_management_smartlist
AutoML timeout: 60 minutes
Sample size: 100,000 records


## Import Libraries


In [0]:
# Get target products from configuration
target_products = config.target_products

print(f"Target products from config: {len(target_products)} models to train")
for i, product in enumerate(target_products, 1):
    print(f"  {i}. {product}")


Target products from config: 4 models to train
  1. investment_cross_sell
  2. retirement_cross_sell
  3. life_insurance_cross_sell
  4. network_products_cross_sell


## Load Training Data
Load the training data from Notebook 01 containing engineered features


In [0]:
# Prepare training data using robust pipeline
print("Preparing training data with feature engineering...")

try:
    # Use the new robust data preparation pipeline
    train_df, test_df = prepare_training_data(
        target_schema=config.target_schema,
        spark=spark,
        train_test_split=config.train_test_split,
        random_seed=config.random_seed
    )
    
    # Validate target columns exist
    validate_target_columns(train_df, target_products)
    
    # Perform data quality validation
    required_columns = target_products + ['client_age', 'acct_val_amt', 'channel']
    validation_results = validate_data_quality(train_df, required_columns)
    
    if not validation_results['is_valid']:
        raise ValueError(f"Data quality validation failed: {validation_results['errors']}")
    
    # Validate target distributions
    target_dist = validate_target_distribution(train_df, target_products)
    if target_dist['warnings']:
        print("Target distribution warnings:")
        for warning in target_dist['warnings']:
            print(f"  - {warning}")
    
    print("Data preparation completed successfully")
    
except Exception as e:
    print(f"ERROR: Failed to prepare training data: {str(e)}")
    raise


Preparing training data with feature engineering...
Starting data preparation pipeline...
Loaded 297,376 records from base table: eda_smartlist.us_wealth_management_smartlist.multi_product_training_data
Base data columns: 108
Adding interaction features...
Successfully added 4 interaction features:
  - wealth_risk_score: account value × stock allocation
  - age_investment_fit: age-adjusted investment propensity
  - relationship_strength: tenure × channel quality
  - wealth_concentration: account value / total assets
Converting DecimalType columns to DoubleType for AutoML compatibility...
Converted 0 DecimalType columns to DoubleType
Splitting data: 80.0% train, 20.0% test
Data preparation completed successfully:
  - Training set: 237,935 records (80.0%)
  - Test set: 59,441 records (20.0%)
  - Total features: 108
Validated all 4 target columns exist in data
Checking data quality...
Data quality validation completed:
  - Total rows: 237,935
  - Total columns: 108
  - Duplicate rows: 0
 

## Data Type Preparation
Convert decimal types to double for compatibility with Delta Lake and AutoML


In [0]:
# Data type conversion is handled automatically by the feature engineering module
print("Data types are properly handled by prepare_training_data()")
print(f"Train data ready: {train_df.count():,} records, {len(train_df.columns)} columns")
print(f"Test data ready: {test_df.count():,} records, {len(test_df.columns)} columns")


Data types are properly handled by prepare_training_data()
Train data ready: 237,935 records, 108 columns
Test data ready: 59,441 records, 108 columns


## Save Training Splits
Save the prepared training and test data to Delta tables for downstream use


In [0]:
# Save training splits to Delta tables for downstream notebooks
print("Saving training splits...")

# Define table names using configuration
train_table = config.train_table
test_table = config.test_table

# Drop existing tables to prevent schema conflicts
print("Dropping existing train/test tables (if any)...")
spark.sql(f"DROP TABLE IF EXISTS {train_table}")
spark.sql(f"DROP TABLE IF EXISTS {test_table}")
print("Existing tables dropped")

# Save train/test data (base table remains immutable)
print("Saving train/test data to Delta tables...")
train_df.write.format("delta").mode("overwrite").saveAsTable(train_table)
test_df.write.format("delta").mode("overwrite").saveAsTable(test_table)

print(f"Training data saved to: {train_table}")
print(f"Test data saved to: {test_table}")
print(f"Base table remains immutable: {config.base_table}")
print(f"Ready for AutoML training with {config.automl_sample_size:,} record sample")


Saving training splits...
Dropping existing train/test tables (if any)...
Existing tables dropped
Saving train/test data to Delta tables...
Training data saved to: eda_smartlist.us_wealth_management_smartlist.multi_product_automl_train
Test data saved to: eda_smartlist.us_wealth_management_smartlist.multi_product_automl_test
Base table remains immutable: eda_smartlist.us_wealth_management_smartlist.multi_product_training_data
Ready for AutoML training with 100,000 record sample


## Systematic Feature Selection
Use data-driven approach to select optimal features for each product


In [0]:
# Train/test split is now handled by prepare_training_data() function
print(f"Train/test split already completed:")
print(f"  - Training records: {train_df.count():,}")
print(f"  - Test records: {test_df.count():,}")


Train/test split already completed:
  - Training records: 237,935
  - Test records: 59,441


## Save Train/Test Data
Save to Delta tables for reproducibility


In [0]:
# Data saving is already handled by the previous cell
# Base table remains immutable, train/test splits are saved
print("Data saving completed in previous cell")
print(f"Ready for feature selection and model training")


Data saving completed in previous cell
Ready for feature selection and model training


## Systematic Feature Selection

Use data-driven approach with data leakage prevention
Select features using data-driven approach combining statistical analysis and domain knowledge


In [0]:
# ============================================================================
# DATA LEAKAGE EXCLUSION LIST - DEFINITIVE
# ============================================================================
# This list contains ALL features that are definitively data leakage
# Based on how features were created in Notebooks 01 and 02:
# - All LEAD() window functions showing NEXT product information
# - All features using cross_sell_product_category (the answer)
# - All target variables (can't predict themselves)
# ============================================================================

DATA_LEAKAGE_EXCLUSION_LIST = [
    # ===== ADMINISTRATIVE/IDENTIFIERS =====
        'axa_party_id', 'policy_no', 'register_date', 'business_month',
        'snapshot_month', 'snapshot_date', 'rnk',
        
        # ===== ALL TARGET VARIABLES (can't use to predict themselves) =====
        'investment_cross_sell', 'retirement_cross_sell',
        'life_insurance_cross_sell', 'network_products_cross_sell',
        'disability_cross_sell', 'health_cross_sell',
        'retirement_ira_cross_sell', 'retirement_401k_cross_sell', 'retirement_403b_cross_sell',
        'life_term_cross_sell', 'life_whole_cross_sell', 'life_universal_cross_sell',
        'investment_advisory_cross_sell', 'investment_annuities_cross_sell',
        'investment_brokerage_cross_sell', 'annuity_cross_sell',
        'network_banking_cross_sell', 'network_mortgage_cross_sell',
        
        # ===== CRITICAL: FUTURE INFORMATION (DATA LEAKAGE) =====
        # These are LEAD() window functions showing what was purchased NEXT
        'cross_sell_product_category',      # THE ANSWER! (investment/retirement/life/network)
        'cross_sell_retirement_sub',        # Sub-category of NEXT retirement product
        'cross_sell_life_sub',              # Sub-category of NEXT life product
        'cross_sell_investment_sub',        # Sub-category of NEXT investment product
        'cross_sell_network_sub',           # Sub-category of NEXT network product
        'cross_sell_account_type',          # Account type of NEXT product
        'cross_sell_tax_treatment',         # Tax treatment of NEXT product
        
        # LEAD() fields (peek into the future)
        'product_lead', 'prod_lob_lead', 'sub_product_level_1_lead',
        'sub_product_level_2_lead', 'policy_no_lead', 'register_date_lead',
        'trmn_eff_date_lead',
        
        # ===== INTERACTION FEATURES WITH LEAKED INFO =====
        # These were computed using cross_sell_product_category
        'life_to_retirement_pattern', 'life_to_investment_pattern',
        'retirement_to_investment_pattern', 'retirement_to_life_pattern',
        'investment_to_retirement_pattern', 'investment_to_life_pattern',
        'senior_retirement_target', 'senior_life_target', 'senior_investment_target',
        'midcareer_retirement_target', 'midcareer_life_target', 'midcareer_investment_target',
        'young_retirement_target', 'young_investment_target',
        'high_aum_investment_target', 'high_aum_retirement_target',
        'ultra_high_aum_network_target', 'medium_aum_life_target',
        'medium_aum_retirement_target', 'branch_to_retirement', 'branch_to_life',
        'advisor_to_investment', 'advisor_to_retirement',
        
        # ===== ADDITIONAL LEAKAGE FEATURES =====
        # All features that use cross_sell_product_category (from Notebook 02)
        # Pattern features
        'life_to_retirement_pattern', 'life_to_investment_pattern',
        'retirement_to_investment_pattern', 'retirement_to_life_pattern',
        'investment_to_retirement_pattern', 'investment_to_life_pattern',
        
        # Target features (age × product)
        'senior_retirement_target', 'senior_life_target', 'senior_investment_target',
        'midcareer_retirement_target', 'midcareer_life_target', 'midcareer_investment_target',
        'young_retirement_target', 'young_investment_target',
        
        # AUM × product targets
        'high_aum_investment_target', 'high_aum_retirement_target',
        'ultra_high_aum_network_target', 'medium_aum_life_target',
        'medium_aum_retirement_target',
        
        # Channel × product
        'branch_to_retirement', 'branch_to_life',
        'advisor_to_investment', 'advisor_to_retirement',
        
        # Market × product demand
        'volatile_market_retirement_demand', 'volatile_market_life_demand',
        'stable_market_investment_demand', 'stable_growth_investment_demand',
        
        # Investor profile × product
        'aggressive_to_investment', 'aggressive_to_network',
        'conservative_to_retirement', 'conservative_to_life',
        
        # Trigger × product
        'retirement_trigger_to_retirement', 'retirement_trigger_to_investment',
        'family_trigger_to_life', 'family_trigger_to_retirement',
        'wealth_trigger_to_investment', 'wealth_trigger_to_retirement',
        
        # Timing features
        'quick_to_investment', 'quick_to_retirement',
        'days_to_cross_sell',  # Uses register_date_lead
        
        # Ideal candidate features
        'ideal_retirement_candidate', 'ideal_investment_candidate',
        'ideal_life_candidate', 'ideal_network_candidate',
        
        # Tenure × product affinity
        'long_tenure_retirement_affinity', 'long_tenure_investment_affinity',
        'new_client_investment_affinity', 'new_client_life_affinity',
        
        # AUM segment × product
        'ultra_high_to_network', 'ultra_high_to_investment',
        'high_to_retirement', 'medium_to_life',
        
        # 3-way interactions with cross_sell_product_category
        'volatile_aggressive_investment', 'stable_conservative_retirement',
        'growth_retirement_planning', 'volatile_family_life',
        
        # Additional cross_sell features
        'cross_sell_investment_type', 'cross_sell_tax_treatment',
        'register_date_dt_lead'  # Future date information
    ]

def systematic_feature_selection(df, target_col, min_features=15, max_features=100):
    """
    Systematic feature selection based on:
    1. Correlation with target (70% weight)
    2. Domain knowledge (30% weight)
    3. CRITICAL: Exclude all data leakage features using DATA_LEAKAGE_EXCLUSION_LIST
    
    No hard cap on features - use as many legitimate features as help the model
    """
    
    print(f"\n{'='*80}")
    print(f"FEATURE SELECTION: {target_col}")
    print(f"{'='*80}")
    
    # Use the shared exclusion list
    exclude_cols = DATA_LEAKAGE_EXCLUSION_LIST
    
    print(f"\nExcluding {len(exclude_cols)} features to prevent data leakage")
    print("  - cross_sell_* features (contain the answer)")
    print("  - *_lead features (future product information)")
    print("  - All target variables")
    print("  - All features using cross_sell_product_category")
    
    feature_candidates = [c for c in df.columns if c not in exclude_cols]
    
    print(f"\nFeature pool: {len(feature_candidates)} candidates")
    
    # Method 1: Compute correlations with target
    print(f"\n1. Computing correlations with {target_col}...")
    correlations = []
    
    # Sample for faster computation
    sample_size = min(50000, df.count())
    sample_df = df.sample(fraction=sample_size/df.count(), seed=42).toPandas()
    
    for feat in feature_candidates:
        try:
            if feat in sample_df.columns:
                # Handle categorical features
                if sample_df[feat].dtype == 'object':
                    sample_df[feat + '_coded'] = pd.Categorical(sample_df[feat]).codes
                    corr_val = sample_df[feat + '_coded'].corr(sample_df[target_col])
                else:
                    corr_val = sample_df[feat].corr(sample_df[target_col])
                
                correlations.append({
                    'feature': feat,
                    'correlation': abs(corr_val) if not pd.isna(corr_val) else 0,
                    'direction': 'positive' if corr_val > 0 else 'negative'
                })
        except:
            pass
    
    correlations_df = pd.DataFrame(correlations).sort_values('correlation', ascending=False)
    
    # Method 2: Domain knowledge scoring
    print(f"\n2. Applying domain knowledge scoring...")
    
    # Define business-relevant feature groups
    domain_scores = {
        'wealth_indicators': ['acct_val_amt', 'aum_segment', 'wc_total_assets', 
                             'wc_net_worth', 'household_aum'],
        'risk_profile': ['wc_assetmix_stocks', 'wc_assetmix_bonds', 'wc_assetmix_mutual_funds',
                        'aggressive_investor', 'conservative_investor'],
        'demographics': ['client_age', 'client_tenure_years', 'channel', 'zip_code_income_median'],
        'product_holdings': ['face_amt', 'monthly_preminum_amount', 'cash_val_amt', 
                            'product_category', 'product_count'],
        'behavioral': ['has_recent_transaction', 'transaction_frequency', 'policy_changes',
                      'contact_frequency', 'digital_engagement'],
        'market_timing': ['snp_close_lead_6', 'snp_close_lead_12', 'snp_close_variation'],
        'life_stage': ['retirement_planning_trigger', 'wealth_building_trigger', 
                       'family_protection_trigger'],
        'affinity': ['advisor_investment_affinity', 'branch_retirement_affinity'],
        'interactions': ['wealth_risk_score', 'age_investment_fit', 'relationship_strength',
                        'wealth_concentration']
    }
    
    # Add domain score
    def get_domain_score(feat):
        for group, features in domain_scores.items():
            if any(f in feat for f in features):
                return 1.0
        return 0.0
    
    correlations_df['domain_score'] = correlations_df['feature'].apply(get_domain_score)
    correlations_df['combined_score'] = (
        correlations_df['correlation'] * 0.7 + 
        correlations_df['domain_score'] * 0.3
    )
    
    # Sort by combined score
    correlations_df = correlations_df.sort_values('combined_score', ascending=False)
    
    # Select features - no arbitrary cap, use all features with meaningful contribution
    # Only limit to max_features if we have too many
    if len(correlations_df) > max_features:
        selected_features = correlations_df.head(max_features)['feature'].tolist()
    else:
        # Use all features with correlation > 0.01 or domain score > 0
        mask = (correlations_df['correlation'] > 0.01) | (correlations_df['domain_score'] > 0)
        selected_features = correlations_df[mask]['feature'].tolist()
    
    # Ensure product-specific must-have features
    must_have_features = []
    if 'investment' in target_col:
        must_have_features = ['acct_val_amt', 'aum_segment', 'wc_assetmix_stocks', 
                             'client_age', 'channel', 'wealth_risk_score']
    elif 'retirement' in target_col:
        must_have_features = ['client_age', 'acct_val_amt', 'wc_assetmix_bonds',
                             'retirement_planning_trigger', 'channel']
    elif 'life_insurance' in target_col:
        must_have_features = ['client_age', 'face_amt', 'monthly_preminum_amount',
                             'family_protection_trigger']
    elif 'network' in target_col:
        must_have_features = ['acct_val_amt', 'aum_segment', 'channel', 
                             'client_tenure_years']
    
    # Add must-have features if missing
    for feat in must_have_features:
        if feat in feature_candidates and feat not in selected_features:
            selected_features.insert(0, feat)
    
    # Ensure minimum features
    if len(selected_features) < min_features:
        remaining = [f for f in correlations_df['feature'] if f not in selected_features]
        selected_features.extend(remaining[:min_features - len(selected_features)])
    
    print(f"\n3. Selected {len(selected_features)} features")
    print(f"\nTop 15 features by combined score:")
    top_features = correlations_df.head(15)
    for idx, row in top_features.iterrows():
        print(f"  {row['feature']:40s} | Corr: {row['correlation']:.4f} | Domain: {row['domain_score']:.1f} | Score: {row['combined_score']:.4f}")
    
    return selected_features

# Generate systematic feature sets for each product
print("\n" + "="*80)
print("GENERATING SYSTEMATIC FEATURE SETS")
print("="*80)

SYSTEMATIC_FEATURE_SETS = {}

# Allow flexible feature counts - models will use what they need
for target in target_products:
    selected_features = systematic_feature_selection(
        train_df, 
        target, 
        min_features=15,   # Minimum to ensure basic model
        max_features=100   # Maximum to prevent overfitting on noise
    )
    SYSTEMATIC_FEATURE_SETS[target] = selected_features

print("\n" + "="*80)
print("FEATURE SELECTION COMPLETE")
print("="*80)
print("\nFeature counts per product:")
for product, features in SYSTEMATIC_FEATURE_SETS.items():
    print(f"  {product}: {len(features)} features")

# Validate feature sets are not empty
if not SYSTEMATIC_FEATURE_SETS:
    raise ValueError("ERROR: No feature sets were generated. Cannot proceed with training.")
    
empty_feature_sets = [product for product, features in SYSTEMATIC_FEATURE_SETS.items() if not features or len(features) == 0]
if empty_feature_sets:
    raise ValueError(f"ERROR: Empty feature sets found for: {empty_feature_sets}. Cannot proceed with training.")

print("\nValidation: All feature sets are populated and ready for training.")



GENERATING SYSTEMATIC FEATURE SETS

FEATURE SELECTION: investment_cross_sell

Excluding 121 features to prevent data leakage
  - cross_sell_* features (contain the answer)
  - *_lead features (future product information)
  - All target variables
  - All features using cross_sell_product_category

Feature pool: 75 candidates

1. Computing correlations with investment_cross_sell...

2. Applying domain knowledge scoring...

3. Selected 63 features

Top 15 features by combined score:
  product_category                         | Corr: 0.4981 | Domain: 1.0 | Score: 0.6487
  aum_segment                              | Corr: 0.2803 | Domain: 1.0 | Score: 0.4962
  retirement_planning_trigger              | Corr: 0.1328 | Domain: 1.0 | Score: 0.3929
  channel                                  | Corr: 0.1255 | Domain: 1.0 | Score: 0.3878
  family_protection_trigger                | Corr: 0.1249 | Domain: 1.0 | Score: 0.3874
  client_age                               | Corr: 0.0854 | Domain: 1.0 | 

## Data Leakage Verification

Verify no leakage features in selected feature sets


In [0]:
# Verify No Data Leakage in Selected Features
# DEFINITIVE CHECK: Verify that NO selected features are in the exclusion list
print("=" * 80)
print("DATA LEAKAGE VERIFICATION")
print("=" * 80)

# Use the same exclusion list as feature selection (ensures consistency)
exclude_set = set(DATA_LEAKAGE_EXCLUSION_LIST)
leakage_found = False

for target, features in SYSTEMATIC_FEATURE_SETS.items():
    print(f"\n{target}:")
    
    # Check if any selected feature is in the exclusion list
    leakage_features = [f for f in features if f in exclude_set]
    
    if leakage_features:
        print(f"  ERROR: Found {len(leakage_features)} leakage features in selection:")
        for leak_feat in leakage_features:
            print(f"    - {leak_feat}")
        leakage_found = True
    else:
        print(f"  VERIFIED: No leakage features ({len(features)} legitimate features)")

# Print summary
print("\n" + "="*80)
if leakage_found:
    print("ERROR: Data leakage features detected in selected feature sets")
    print("This should not happen - the exclusion list should prevent this.")
    print("Training will proceed, but models may show unrealistic performance (ROC-AUC > 0.95)")
    print("Review the feature selection logic if this occurs.")
else:
    print("SUCCESS: All feature sets verified - No leakage features detected")
    print("All selected features are legitimate and safe for training")
print("="*80)
print("\nProceeding with AutoML training...")


DATA LEAKAGE VERIFICATION

investment_cross_sell:
  VERIFIED: No leakage features (63 legitimate features)

retirement_cross_sell:
  VERIFIED: No leakage features (61 legitimate features)

life_insurance_cross_sell:
  VERIFIED: No leakage features (58 legitimate features)

network_products_cross_sell:
  VERIFIED: No leakage features (55 legitimate features)

SUCCESS: All feature sets verified - No leakage features detected
All selected features are legitimate and safe for training

Proceeding with AutoML training...


## AutoML Training
Train 4 independent models using AutoML with systematically selected features

In [0]:
# AutoML Training Loop
automl_config = {
    'experiment_base_dir': f'/Users/{spark.sql("SELECT current_user()").collect()[0][0]}/databricks_automl',
    'timeout_minutes': config.automl_timeout_minutes
}

training_results = {}

print("=" * 80)
print("STARTING AUTOML TRAINING")
print("=" * 80)
print(f"\nConfiguration:")
print(f"  Timeout per model: {config.automl_timeout_minutes} minutes")
print(f"  Training sample size: {config.automl_sample_size:,} records")
print(f"  Models to train: {len(target_products)}")
print(f"  Expected total time: {config.automl_timeout_minutes * len(target_products) / 60:.1f} hours\n")

for idx, target in enumerate(target_products, 1):
    print("\n" + "=" * 80)
    print(f"MODEL {idx}/{len(target_products)}: {target.upper()}")
    print("=" * 80)
    
    try:
        # Get systematically selected features
        feature_cols = SYSTEMATIC_FEATURE_SETS[target]
        
        # Validate features exist in train_df
        missing_features = [f for f in feature_cols if f not in train_df.columns]
        if missing_features:
            raise ValueError(f"ERROR: Selected features not found in data: {missing_features[:5]}... (showing first 5)")
        
        print(f"\nFeature Set:")
        print(f"  Count: {len(feature_cols)}")
        print(f"  Features: {', '.join(feature_cols[:10])}...")
        
        # Validate target exists
        if target not in train_df.columns:
            raise ValueError(f"ERROR: Target column '{target}' not found in training data.")
        
        # Check target distribution
        target_dist = train_df.groupBy(target).count().collect()
        total_count = sum([row['count'] for row in target_dist])
        
        # Handle case where positive_count might be 0
        positive_rows = [row for row in target_dist if row[target] == 1]
        if not positive_rows:
            print(f"\nWARNING: No positive samples found for {target}. Skipping this model.")
            continue
        
        positive_count = positive_rows[0]['count']
        if positive_count == 0:
            print(f"\nWARNING: Zero positive samples for {target}. Skipping this model.")
            continue
            
        positive_rate = positive_count / total_count
        
        print(f"\nTarget Distribution:")
        print(f"  Positive samples: {positive_count:,} ({positive_rate*100:.1f}%)")
        print(f"  Total samples: {total_count:,}")
        
        # Prepare AutoML data
        columns_to_select = feature_cols + [target]
        
        # Check for missing features
        missing_features = [f for f in columns_to_select if f not in train_df.columns]
        if missing_features:
            print(f"\nWARNING: Missing features: {missing_features}")
            print(f"Removing missing features from training...")
            columns_to_select = [f for f in columns_to_select if f in train_df.columns]
            feature_cols = [f for f in feature_cols if f in train_df.columns]
        
        # Sample data for AutoML
        automl_data = train_df.select(columns_to_select).sample(
            fraction=min(1.0, config.automl_sample_size / total_count),
            seed=42
        )
        
        actual_sample_size = automl_data.count()
        print(f"\nAutoML Training Data:")
        print(f"  Sample size: {actual_sample_size:,} records")
        print(f"  Features: {len(feature_cols)}")
        print(f"  Target: {target}")
        
        # Run AutoML
        print(f"\nStarting AutoML training...")
        print(f"  Expected completion: {datetime.now().strftime('%H:%M:%S')} + {config.automl_timeout_minutes} min")
        
        summary = automl.classify(
            dataset=automl_data,
            target_col=target,
            primary_metric='roc_auc',
            timeout_minutes=config.automl_timeout_minutes,
            pos_label=1,
            experiment_dir=f"{automl_config['experiment_base_dir']}/{target}_{business_month}"
        )
        
        # Extract transformed feature information
        print(f"\nExtracting model metadata...")
        transformed_feature_names = None
        transformed_feature_count = None
        
        try:
            best_model_uri = f"runs:/{summary.best_trial.mlflow_run_id}/model"
            model = mlflow.sklearn.load_model(best_model_uri)
            preprocessing_pipeline = model[:-1]
            
            # Get sample for transformation
            sample_pdf = train_df.select(feature_cols).limit(1).toPandas()
            X_transformed = preprocessing_pipeline.transform(sample_pdf)
            
            if hasattr(X_transformed, 'toarray'):
                X_transformed = X_transformed.toarray()
            
            transformed_feature_count = X_transformed.shape[1]
            
            try:
                transformed_feature_names = list(preprocessing_pipeline.get_feature_names_out(feature_cols))
            except:
                try:
                    transformed_feature_names = list(preprocessing_pipeline.get_feature_names_out())
                except:
                    transformed_feature_names = [f"feature_{i}" for i in range(transformed_feature_count)]
            
            print(f"  Transformed features: {transformed_feature_count}")
            
        except Exception as e:
            print(f"  Could not extract transformed features: {str(e)}")
        
        # Store results (mark as successful)
        training_results[target] = {
            'summary': summary,
            'features': feature_cols,
            'feature_count': len(feature_cols),
            'transformed_features': transformed_feature_names,
            'transformed_feature_count': transformed_feature_count,
            'positive_rate': positive_rate,
            'sample_size': actual_sample_size,
            'success': True
        }
        
        roc_auc = summary.best_trial.metrics.get('val_roc_auc', 
                    summary.best_trial.metrics.get('roc_auc', 0.0))
        
        print(f"\nAutoML COMPLETED for {target}")
        print(f"  Best model: {summary.best_trial.model_description}")
        print(f"  Best ROC-AUC: {roc_auc:.4f}")
        print(f"  Experiment ID: {summary.experiment.experiment_id}")
        
    except Exception as e:
        print(f"\n{'='*80}")
        print(f"ERROR training {target}:")
        print(f"  {str(e)}")
        print(f"{'='*80}")
        print(f"Continuing with next model...\n")
        
        # Store error info but don't mark as successful
        training_results[target] = {
            'error': str(e),
            'success': False,
            'features': feature_cols if 'feature_cols' in locals() else [],
            'feature_count': len(feature_cols) if 'feature_cols' in locals() else 0
        }
        continue

print("\n" + "=" * 80)
print("AUTOML TRAINING COMPLETE")
print("=" * 80)


STARTING AUTOML TRAINING

Configuration:
  Timeout per model: 60 minutes
  Training sample size: 100,000 records
  Models to train: 4
  Expected total time: 4.0 hours


MODEL 1/4: INVESTMENT_CROSS_SELL

Feature Set:
  Count: 63
  Features: product_category, aum_segment, retirement_planning_trigger, channel, family_protection_trigger, client_age, snp_close_lead_12, acct_val_amt, wealth_building_trigger, wc_total_assets...

Target Distribution:
  Positive samples: 155,042 (65.2%)
  Total samples: 237,935

AutoML Training Data:
  Sample size: 99,908 records
  Features: 63
  Target: investment_cross_sell

Starting AutoML training...
  Expected completion: 14:51:48 + 60 min


2025/11/26 14:52:08 INFO databricks.automl.client.manager: AutoML will optimize for ROC/AUC metric, which is tracked as val_roc_auc in the MLflow experiment.
2025/11/26 14:52:10 INFO databricks.automl.client.manager: MLflow Experiment ID: 1506772837420558
2025/11/26 14:52:10 INFO databricks.automl.client.manager: MLflow Experiment: https://adb-8916084915459926.6.azuredatabricks.net/?o=8916084915459926#mlflow/experiments/1506772837420558
2025/11/26 14:54:57 INFO databricks.automl.client.manager: Data exploration notebook: https://adb-8916084915459926.6.azuredatabricks.net/?o=8916084915459926#notebook/1506772837420563
2025/11/26 15:53:17 INFO databricks.automl.client.manager: AutoML experiment completed successfully.


,Train,Validation,Test
roc_auc,0.950,0.903,0.905
recall_score,0.940,0.906,0.907
false_negatives,2363.000,1217.000,1217.000
false_positives,4962.000,2061.000,2095.000
example_count,60055.000,19799.000,20054.000
precision_score,0.881,0.850,0.850
true_positives,36792.000,11696.000,11832.000
precision_recall_auc,0.972,0.942,0.942
true_negatives,15938.000,4825.000,4910.000
log_loss,0.286,0.367,0.364



Extracting model metadata...


  Transformed features: 530

AutoML COMPLETED for investment_cross_sell
  Best model: LGBMClassifier(colsample_bytree=0.7844859970455422,
               lambda_l1=0.35876178893210753, lambda_l2=0.517384101430882,
               learning_rate=0.01896090670683733, max_bin=312, max_depth=12,
               min_child_samples=44, n_estimators=668, num_leaves=569,
               path_smooth=24.29090820914173, random_state=97895661,
               subsample=0.7001098211630928)
  Best ROC-AUC: 0.9034
  Experiment ID: 1506772837420558

MODEL 2/4: RETIREMENT_CROSS_SELL

Feature Set:
  Count: 61
  Features: product_category, aum_segment, retirement_planning_trigger, channel, face_amt, family_protection_trigger, cash_val_amt, wc_total_assets, snp_close_lead_12, wc_assetmix_stocks...

Target Distribution:
  Positive samples: 60,014 (25.2%)
  Total samples: 237,869

AutoML Training Data:
  Sample size: 99,909 records
  Features: 61
  Target: retirement_cross_sell

Starting AutoML training...
  Expec

...

  Transformed features: 508

AutoML COMPLETED for life_insurance_cross_sell
  Best model: LGBMClassifier(colsample_bytree=0.5540938437079953,
               lambda_l1=0.2768916363253556, lambda_l2=0.45176799297213616,
               learning_rate=0.039953180972397244, max_bin=305, max_depth=8,
               min_child_samples=29, n_estimators=568, num_leaves=483,
               path_smooth=46.085734540611355, random_state=729957982,
               subsample=0.767237435453763)
  Best ROC-AUC: 0.8901
  Experiment ID: 1506772837420574

MODEL 4/4: NETWORK_PRODUCTS_CROSS_SELL

Feature Set:
  Count: 55
  Features: client_age, cash_val_amt, aum_segment, snp_close_lead_12, face_amt, product_category, aggressive_investor, acct_val_amt, wealth_risk_score, snp_close_variation...

Target Distribution:
  Positive samples: 10,671 (4.5%)
  Total samples: 237,935

AutoML Training Data:
  Sample size: 99,908 records
  Features: 55
  Target: network_products_cross_sell

Starting AutoML training...
  Exp

2025/11/26 17:51:52 INFO databricks.automl.client.manager: AutoML will optimize for ROC/AUC metric, which is tracked as val_roc_auc in the MLflow experiment.
2025/11/26 17:51:53 INFO databricks.automl.client.manager: MLflow Experiment ID: 1506772837420581
2025/11/26 17:51:53 INFO databricks.automl.client.manager: MLflow Experiment: https://adb-8916084915459926.6.azuredatabricks.net/?o=8916084915459926#mlflow/experiments/1506772837420581
2025/11/26 17:53:48 INFO databricks.automl.client.manager: Data exploration notebook: https://adb-8916084915459926.6.azuredatabricks.net/?o=8916084915459926#notebook/1506772837420586
2025/11/26 18:52:36 INFO databricks.automl.client.manager: AutoML experiment completed successfully.


,Train,Validation,Test
roc_auc,0.933,0.824,0.831
recall_score,0.197,0.158,0.170
false_negatives,2138.000,744.000,727.000
false_positives,62.000,62.000,49.000
example_count,42022.000,19799.000,20054.000
precision_score,0.894,0.693,0.753
true_positives,524.000,140.000,149.000
precision_recall_auc,0.604,0.330,0.356
true_negatives,39298.000,18853.000,19129.000
log_loss,0.147,0.142,0.137



Extracting model metadata...


  Transformed features: 452

AutoML COMPLETED for network_products_cross_sell
  Best model: LGBMClassifier(colsample_bytree=0.5271310238727839,
               lambda_l1=0.12138925614569594, lambda_l2=0.117691126104428,
               learning_rate=0.02736254677773714, max_bin=461, max_depth=9,
               min_child_samples=38, n_estimators=451, num_leaves=647,
               path_smooth=51.88493392603752, random_state=183514515,
               subsample=0.6769134242386495)
  Best ROC-AUC: 0.8244
  Experiment ID: 1506772837420581

AUTOML TRAINING COMPLETE


In [0]:
# Training Summary
print("\n" + "=" * 80)
print("TRAINING SUMMARY")
print("=" * 80)

for target, result in training_results.items():
    print(f"\n{target.upper()}:")
    print(f"  Features used: {result['feature_count']}")
    
    if 'error' in result:
        print(f"  Status: FAILED")
        print(f"  Error: {result['error']}")
    else:
        roc_auc = result['summary'].best_trial.metrics.get('val_roc_auc', 
                    result['summary'].best_trial.metrics.get('roc_auc', 0.0))
        print(f"  Status: SUCCESS")
        print(f"  Best model: {result['summary'].best_trial.model_description}")
        print(f"  ROC-AUC: {roc_auc:.4f}")
        print(f"  Positive rate: {result['positive_rate']*100:.1f}%")
        print(f"  Sample size: {result['sample_size']:,}")

print("\n" + "=" * 80)
print("ALL TRAINING COMPLETE")
print("=" * 80)



TRAINING SUMMARY

INVESTMENT_CROSS_SELL:
  Features used: 63
  Status: SUCCESS
  Best model: LGBMClassifier(colsample_bytree=0.7844859970455422,
               lambda_l1=0.35876178893210753, lambda_l2=0.517384101430882,
               learning_rate=0.01896090670683733, max_bin=312, max_depth=12,
               min_child_samples=44, n_estimators=668, num_leaves=569,
               path_smooth=24.29090820914173, random_state=97895661,
               subsample=0.7001098211630928)
  ROC-AUC: 0.9034
  Positive rate: 65.2%
  Sample size: 99,908

RETIREMENT_CROSS_SELL:
  Features used: 61
  Status: SUCCESS
  Best model: TransformedTargetClassifier(classifier=XGBClassifier(base_score=None,
                                                     booster=None,
                                                     callbacks=None,
                                                     colsample_bylevel=None,
                                                     colsample_bynode=None,
                   

## Register Models to Unity Catalog

Register all trained models to Unity Catalog for easy access in prediction notebooks


In [0]:
# Get business month from widget
business_month = dbutils.widgets.get("business_month")

# Register Models to Unity Catalog
print("=" * 80)
print("REGISTERING MODELS TO UNITY CATALOG")
print("=" * 80)
mlflow.set_registry_uri("databricks-uc")
registered_models = {}
for target, result in training_results.items():
    if 'error' not in result:
        print(f"\nRegistering {target}...")
        try: 
            experiment_id = result['summary'].experiment.experiment_id
            best_trial = result['summary'].best_trial
            run_id = best_trial.mlflow_run_id
            model_name = f"eda_smartlist.models.{target}_{business_month}"
            model_uri = f"runs:/{run_id}/model"
            model_version = mlflow.register_model(model_uri=model_uri, name=model_name)
            roc_auc = result['summary'].best_trial.metrics.get('val_roc_auc',
                result['summary'].best_trial.metrics.get('roc_auc', 0.0))
            registered_models[target] = {                
                'model_name': model_name,                
                'version': model_version.version,                
                'run_id': run_id,                
                'roc_auc': roc_auc,                
                'feature_count': result['feature_count'],                
                'features': result['features'],                
                'transformed_features': result.get('transformed_features'),                
                'transformed_feature_count': result.get('transformed_feature_count')            
            }                        
            print(f"   Registered: {model_name}")            
            print(f"   Version: {model_version.version}")            
            print(f"   ROC-AUC: {roc_auc:.4f}")        
        except Exception as e:            
            print(f"   Error: {str(e)}")
        print("\n" + "=" * 80)
        print("Models ready for production use!")
        print(f"Load with: mlflow.sklearn.load_model('models:/eda_smartlist.models.{{product}}_{business_month}/1')")

REGISTERING MODELS TO UNITY CATALOG

Registering investment_cross_sell...


Registered model 'eda_smartlist.models.investment_cross_sell_202510' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Created version '3' of model 'eda_smartlist.models.investment_cross_sell_202510'.


   Registered: eda_smartlist.models.investment_cross_sell_202510
   Version: 3
   ROC-AUC: 0.9034

Models ready for production use!
Load with: mlflow.sklearn.load_model('models:/eda_smartlist.models.{product}_202510/1')

Registering retirement_cross_sell...


...

   Registered: eda_smartlist.models.life_insurance_cross_sell_202510
   Version: 1
   ROC-AUC: 0.8901

Models ready for production use!
Load with: mlflow.sklearn.load_model('models:/eda_smartlist.models.{product}_202510/1')

Registering network_products_cross_sell...


Successfully registered model 'eda_smartlist.models.network_products_cross_sell_202510'.


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

   Registered: eda_smartlist.models.network_products_cross_sell_202510
   Version: 1
   ROC-AUC: 0.8244

Models ready for production use!
Load with: mlflow.sklearn.load_model('models:/eda_smartlist.models.{product}_202510/1')


Created version '1' of model 'eda_smartlist.models.network_products_cross_sell_202510'.


## Save Model Metadata

Save model metadata to Unity Catalog for tracking and production reference


In [0]:
# Save Model Metadata for Production
metadata_records = []

for target, info in registered_models.items():
    metadata_records.append({
        'business_month': business_month,
        'product': target,
        'model_name': info['model_name'],
        'model_version': str(info['version']),
        'run_id': info['run_id'],
        'roc_auc': float(info['roc_auc']),
        'feature_count': info['feature_count'],
        'features': ','.join(info['features']),
        'transformed_feature_count': info.get('transformed_feature_count'),
        'transformed_features': ','.join(info['transformed_features']) if info.get('transformed_features') else None,
        'training_date': datetime.now().isoformat(),
        'config.automl_timeout_minutes': config.automl_timeout_minutes,
        'training_sample_size': config.automl_sample_size
    })

metadata_df = spark.createDataFrame(metadata_records)
metadata_table = f"{target_schema}.multi_product_model_metadata"

# Drop existing table to prevent schema conflicts
print(f"Dropping existing metadata table (if any)...")
spark.sql(f"DROP TABLE IF EXISTS {metadata_table}")

# Save metadata
metadata_df.write.mode("overwrite").saveAsTable(metadata_table)

print(f"Model metadata saved to: {metadata_table}")
display(metadata_df)


Dropping existing metadata table (if any)...
Model metadata saved to: eda_smartlist.us_wealth_management_smartlist.multi_product_model_metadata


business_month config.automl_timeout_minutes feature_count features model_name model_version product roc_auc run_id training_date training_sample_size transformed_feature_count transformed_features 202510 60 63 product_category,aum_segment,retirement_planning_trigger,channel,family_protection_trigger,client_age,snp_close_lead_12,acct_val_amt,wealth_building_trigger,wc_total_assets,wc_assetmix_mutual_funds,snp_close_variation,wc_assetmix_stocks,advisor_investment_affinity,wc_assetmix_bonds,face_amt,age_investment_fit,cash_val_amt,branch_retirement_affinity,birth_dt,Product,wealth_risk_score,aggressive_investor,wealth_concentration,monthly_preminum_amount,snp_close_lead_6,sub_product_level_1,conservative_investor,client_tenure_years,relationship_strength,prod_lob,tax_treatment,retirement_sub_category,investment_sub_category,aum_band,snp_low_month,snp_close_month,snp_open_month,snp_high_month,snp_business_month,account_type,client_seg,life_sub_category,retirement_roth_cross_sell,snp_low_lead_3,psn_age,snp_low_variation,snp_low_lead_9,investment_type,snp_low_lead_6,sub_product_level_2,trmn_eff_date,wc_assetmix_deposits,snp_close_lead_9,stock_allocation_ratio,snp_close_lead_3,bond_allocation_ratio,mutual_fund_allocation_ratio,retirement_annuity_cross_sell,wc_assetmix_other_assets,wc_assetmix_annuity,annuity_allocation_ratio,premium_to_aum_ratio eda_smartlist.models.investment_cross_sell_202510 3 investment_cross_sell 0.9033985996095905 7a502f74d5aa4bbeab5f4b28b2745341 2025-11-26T18:53:02.600710 100000 530 feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feat